**Chapter 56 The confusion matrix, prevalence, sensitivity and specificity**

In [ ]:
library(caret)
library(dslabs)
library(dplyr)
data(heights)

We start by defining the outcome and predictors. In this case, we have only one predictor:

In [ ]:
y <- heights$sex
x <- heights$height

This is clearly a categorical outcome since  Y can be Male or Female and we only have one predictor: height. We know that we will not be able to predict  Y very accurately based on  X because male and female average heights are not that different relative to within group variability. But can we do better than guessing? To answer this question, we need a quantitative definition of better.

**56.1 Training and test sets**

Ultimately, a machine learning algorithm is evaluated on how it performs in the real world with completely new datasets. However, when developing an algorithm, we usually have a dataset for which we know the outcomes, as we do with the heights: we know the sex of every student in our dataset. Therefore, to mimic the ultimate evaluation process, we typically split the data into two and act as if we don’t know the outcome for one of these. We stop pretending we don’t know the outcome to evaluate the algorithm, but only after we are done constructing it. We refer to the group for which we know the outcome and use it to develop the algorithm as the training set, and the group for which we pretend we don’t know the outcome as the test set.

A standard way of generating the training and test sets is by randomly splitting the data. The caret package includes the function createDataPartition that helps us generates indexes for randomly splitting the data into training and test sets:

In [ ]:
set.seed(2)
test_index <- createDataPartition(y, times = 1, p = 0.5, list = FALSE)

The argument times is used to define how many random samples of indexes to return, the argument p is used to define what proportion of the data is represented by the index, and the argument list is used to decide if we want the indexes returned as a list or not. We can use the result of the function call to define the training and test sets like this:

In [ ]:
test_set <- heights[test_index, ]
train_set <- heights[-test_index, ]

We will now develop an algorithm using only the training set. Once we are done developing the algorithm, we will freeze it and evaluate it using the test set. The simplest way to evaluate the algorithm when the outcomes are categorical is by simply reporting the proportion of cases that were correctly predicted in the test set. This metric is usually referred to as overall accuracy.

**56.2 Overall accuracy**

To demonstrate the use of overall accuracy, we will build two competing algorithms and compare them.
Let’s start by developing the simplest possible machine algorithm: guessing the outcome.

In [ ]:
y_hat <- sample(c("Male", "Female"), length(test_index), replace = TRUE)

Note that we are completely ignoring the predictor and simply guessing the sex.

In machine learning applications, it is useful to use factors to represent the categorical outcomes because R functions developed for machine learning, such as those in the caret package, require or recommend that categorical outcomes be coded as factors. So convert y_hat to factors using the factor function:

In [ ]:
y_hat <- sample(c("Male", "Female"), length(test_index), replace = TRUE) %>% 
  factor(levels = levels(test_set$sex))

The overall accuracy is simply defined as the overall proportion that is predicted correctly:

In [ ]:
mean(y_hat == test_set$sex)

Not surprisingly, our accuracy is about 50%. We are guessing!

Can we do better? Exploratory data analysis suggests we can because, on average, males are slightly taller than females:

In [ ]:
heights %>% group_by(sex) %>% summarize(mean(height), sd(height))

But how do we make use of this insight? Let’s try another simple approach: predict Male if height is within two standard deviations from the average male:

In [ ]:
y_hat <- ifelse(x > 62, "Male", "Female") %>% factor(levels = levels(test_set$sex))

The accuracy goes up from 0.50 to about 0.80:

In [ ]:
mean(y == y_hat)

But can we do even better? In the example above, we used a cutoff of 62, but we can examine the accuracy obtained for other cutoffs and then pick the value that provides the best results. But remember, it is important that we optimize the cutoff using only the training set: the test set is only for evaluation. Although for this simplistic example it is not much of a problem, later we will learn that evaluating an algorithm on the training set can lead to overfitting, which often results in dangerously over-optimistic assessments.

Here we examine the accuracy of 10 different cutoffs and pick the one yielding the best result:

I just played around with function sapply and map_dbl (map double) and they both give the same outcome
so you can prob use either one of them. 

These are some of the diff in apply functions 
* apply ---> you need to specific margin or which colum or row to apply
* lapply ---> stand for list and give outcome in list but no need to specify margin
* sapply ---> prob  the best of the outcome no margin and come out with vector instead of list

In [ ]:
library(purrr)
cutoff <- seq(61, 70)
accuracy <- map_dbl(cutoff, function(x){
  y_hat <- ifelse(train_set$height > x, "Male", "Female") %>% factor(levels = levels(test_set$sex))
  mean(y_hat == train_set$sex)
})

**Below is some types for plotting**
**type	description**

* p	points
* l	lines
* o	overplotted points and lines
* b, c	points (empty if "c") joined by lines
* s, S	stair steps
* h	histogram-like vertical lines
* n	does not produce any points or lines


In [ ]:

plot(cutoff,accuracy,type='o')



We see that the maximum value is:

In [ ]:
max(accuracy)

which is much higher than 0.5. The cutoff resulting in this accuracy is:

In [ ]:
best_cutoff <- cutoff[which.max(accuracy)]
best_cutoff

Now we can now test this cutoff on our test set to make sure our accuracy is not overly optimistic:

In [ ]:
y_hat <- ifelse(test_set$height > best_cutoff, "Male", "Female") %>% factor(levels = levels(test_set$sex))
y_hat <- factor(y_hat)
mean(y_hat == test_set$sex)

We see that it is a bit lower than the accuracy observed for the training set, but it is still better than guessing. And by testing on a dataset that we did not train on, we know it is not due to cherry-picking a good result.

**56.3 The Confusion Matrix**

The prediction rule we developed in the previous section predicts Male if the student is taller than 64 inches. Given that the average female is about 65 inches, this prediction rule seems wrong. What happened? If a student is the height of the average female, shouldn’t we predict Female?

Generally speaking, overall accuracy can be a deceptive measure. To see this, we will start by constructing what is referred to as the confusion matrix, which basically tabulates each combination of prediction and actual value. We can do this in R using the function table:

In [ ]:
table(predicted = y_hat, actual = test_set$sex)

If we study this table closely, it reveals a problem. If we compute the accuracy separately for each sex, we get:

In [ ]:
test_set %>% 
  mutate(y_hat = y_hat) %>%
  group_by(sex) %>% 
  summarize(accuracy = mean(y_hat == sex))

There is an imbalance in the accuracy for males and females: too many females are predicted to be male. We are calling almost half of the females, males! How can our overall accuracy be so high then? This is because the prevalence of males in this dataset is high. These heights were collected from three data sciences courses, two of which had more males enrolled:

In [ ]:
prev <- mean(y == "Male")
prev

So when computing overall accuracy, the high percentage of mistakes made for females is outweighed by the gains in correct calls for men. This can actually be a big problem in machine learning. If your training data is biased in some way, you are likely to develop algorithms that are biased as well. The fact that we used a test set does not matter because it is also derived from the original, biased dataset. This is one of the reasons we look at metrics other than overall accuracy when evaluating a machine learning algorithm.

There are several metrics that we can use to evaluate an algorithm in a way that prevalence does not cloud our assessment, and these can all be derived from the confusion matrix. A general improvement to using overall accuracy is to study sensitivity and specificity separately.

To get a better understanding of confusion matrix, just check the attached in kaggle under spreadsheet name confusionmatrix



In [ ]:
confusionMatrix(data = y_hat, reference = test_set$sex)

Check out this link which gives great info on confusion matric [great link](https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/)

In [ ]:
#56.5 Balanced accuracy and F1 score

cutoff <- seq(61, 70)
F_1 <- map_dbl(cutoff, function(x){
  y_hat <- ifelse(train_set$height > x, "Male", "Female") %>% factor(levels = levels(test_set$sex))
  F_meas(data = y_hat, reference = factor(train_set$sex))
})

We see that it is maximized at  F1 value of:

In [ ]:
max(F_1)

In [ ]:
best_cutoff <- cutoff[which.max(F_1)]
best_cutoff

A cutoff of 66 makes much more sense than 64. Furthermore, it balances the specificity and sensitivity of our confusion matrix:

In [ ]:
y_hat <- ifelse(test_set$height > best_cutoff, "Male", "Female") %>% factor(levels = levels(test_set$sex))
confusionMatrix(data = y_hat, reference = test_set$sex)

We now see that we do much better than guessing, that both sensitivity and specificity are relatively high, and that we have built our first machine learning algorithm. It takes height as a predictor and predicts female, if you are 66 inches or shorter.

**56.7 ROC and precision-recall curves**

When comparing the two methods: guessing versus using a height cutoff, we looked at accuracy and  F1. The second method clearly outperformed. However, while we considered several cutoffs for the second method, for the first we only considered one approach: guessing with equal probability. Note that guessing Male with higher probability would give us higher accuracy due to the bias in the sample:



In [ ]:
p <- 0.9
y_hat <- sample(c("Male", "Female"), length(test_index), replace = TRUE, prob=c(p, 1-p)) %>% 
  factor(levels = levels(test_set$sex))
mean(y_hat == test_set$sex)

But, as described above, this would come at the cost of lower sensitivity. The curves we describe in this section will help us see this.

Remember that for each of these parameters, we can get a different sensitivity and specificity. For this reason, a very common approach to evaluating methods is to compare them graphically by plotting both.

A widely used plot that does this is the receiver operating characteristic (ROC) curve. If you are wondering where this name comes from, according to Wikipedia:

The ROC curve was first used during World War II for the analysis of radar signals before it was employed in signal detection theory.[35] Following the attack on Pearl Harbor in 1941, the United States army began new research to increase the prediction of correctly detected Japanese aircraft from their radar signals. For this purpose they measured the ability of radar receiver operators to make these important distinctions, which was called the Receiver Operating Characteristics.

The ROC curve plots sensitivity (TPR) versus 1 - specificity or the false positive rate (FPR). Here is an ROC curve for guessing sex but using different probabilities of guessing male:

In [ ]:
probs <- seq(0, 1, length.out = 10)
guessing <- map_df(probs, function(p){
  y_hat <- 
    sample(c("Male", "Female"), length(test_index), replace = TRUE, prob=c(p, 1-p)) %>% 
    factor(levels = c("Female", "Male"))
  list(method = "Guessing",
       FPR = 1 - specificity(y_hat, test_set$sex),
       TPR = sensitivity(y_hat, test_set$sex))
})
guessing %>% qplot(FPR, TPR, data =., xlab = "1 - Specificity", ylab = "Sensitivity")

The ROC curve for guessing always looks like a straight line. A perfect algorithm would shoot straight to 1 and stay up there: perfect sensitivity for all values of specificity. So how does our second approach compare? We can construct an ROC curve for the height based approach:

In [ ]:
cutoffs <- c(50, seq(60, 75), 80)
height_cutoff <- map_df(cutoffs, function(x){
  y_hat <- ifelse(test_set$height > x, "Male", "Female") %>% 
    factor(levels = c("Female", "Male"))
   list(method = "Height cutoff",
        FPR = 1-specificity(y_hat, test_set$sex),
        TPR = sensitivity(y_hat, test_set$sex))
})

In [ ]:
bind_rows(guessing, height_cutoff) %>%
  ggplot(aes(FPR, TPR, color = method)) +
  geom_line() +
  geom_point() +
  xlab("1 - Specificity") +
  ylab("Sensitivity")

In [ ]:
map_df(cutoffs, function(x){
  y_hat <- ifelse(test_set$height > x, "Male", "Female") %>% 
    factor(levels = c("Female", "Male"))
   list(method = "Height cutoff",
        cutoff = x, 
        FPR = 1-specificity(y_hat, test_set$sex),
        TPR = sensitivity(y_hat, test_set$sex))
}) %>%
  ggplot(aes(FPR, TPR, label = cutoff)) +
  geom_line() +
  geom_point() +
  geom_text(nudge_y = 0.01)

In [ ]:
guessing <- map_df(probs, function(p){
  y_hat <- sample(c("Male", "Female"), length(test_index), 
                  replace = TRUE, prob=c(p, 1-p)) %>% 
    factor(levels = c("Female", "Male"))
  list(method = "Guess",
    recall = sensitivity(y_hat, test_set$sex),
    precision = precision(y_hat, test_set$sex))
})

height_cutoff <- map_df(cutoffs, function(x){
  y_hat <- ifelse(test_set$height > x, "Male", "Female") %>% 
    factor(levels = c("Female", "Male"))
  list(method = "Height cutoff",
       recall = sensitivity(y_hat, test_set$sex),
    precision = precision(y_hat, test_set$sex))
})
bind_rows(guessing, height_cutoff) %>%
  ggplot(aes(recall, precision, color = method)) +
  geom_line() +
  geom_point()




In [ ]:
guessing <- map_df(probs, function(p){
  y_hat <- sample(c("Male", "Female"), length(test_index), replace = TRUE, 
                  prob=c(p, 1-p)) %>% 
    factor(levels = c("Male", "Female"))
  list(method = "Guess",
    recall = sensitivity(y_hat, relevel(test_set$sex, "Male", "Female")),
    precision = precision(y_hat, relevel(test_set$sex, "Male", "Female")))
})

height_cutoff <- map_df(cutoffs, function(x){
  y_hat <- ifelse(test_set$height > x, "Male", "Female") %>% 
    factor(levels = c("Male", "Female"))
  list(method = "Height cutoff",
       recall = sensitivity(y_hat, relevel(test_set$sex, "Male", "Female")),
    precision = precision(y_hat, relevel(test_set$sex, "Male", "Female")))
})
bind_rows(guessing, height_cutoff) %>%
  ggplot(aes(recall, precision, color = method)) +
  geom_line() +
  geom_point()

**Chapter 67 Linear regression for prediction**

Linear regression can be considered a machine learning algorithm. As we will see, it is too rigid to be useful in general, but for some challenges it works rather well. It also serves as a baseline approach: if you can’t beat it with a more complex approach, you probably want to stick to linear regression. To quickly make the connection between regression and machine learning, we will reformulate Galton’s study with heights: a continuous outcome.

In [ ]:
library(HistData)

galton_heights <- GaltonFamilies %>%
  filter(childNum == 1 & gender == "male") %>%
  select(father, childHeight) %>%
  rename(son = childHeight)

Suppose you are tasked with building a machine learning algorithm that predicts the son’s height  Y using the father’s height  X. Let’s generate testing and training sets:

In [ ]:
library(caret)
y <- galton_heights$son
test_index <- createDataPartition(y, times = 1, p = 0.5, list = FALSE)

train_set <- galton_heights %>% slice(-test_index)
test_set <- galton_heights %>% slice(test_index)

In [ ]:
head(train_set)

In this case, if we were just ignoring the father’s height and guessing the son’s height we would guess the average height of sons.
And you can see the diff in results as well below.

In [ ]:
avg <- mean(train_set$son)
avg

mean((avg - test_set$son)^2)

The below code showing linear model fitting data to f(x) = b0 + b1x..... in the lm model, intercept is b0 and father is b1

In [ ]:
fit <- lm(son ~ father, data = train_set)
fit$coef

Now you can fix the b0 and b1 to your model using code below and you can see the mean diff has now reduced.

In [ ]:
y_hat <- fit$coef[1] + fit$coef[2]*test_set$father
mean((y_hat - test_set$son)^2)

**67.1 The predict function**

The predict function is very useful for machine learning applications. This function takes a fitted object from function such as lm or glm (we learn about glm soon) and a data frame with the new predictors for which to predict. So in our current example we would use predict like this:

You notice you get the same result as above

In [ ]:
y_hat <- predict(fit, test_set)
mean((y_hat - test_set$son)^2)

Predict does not always return objects of the same types; it depends on what type of object is sent to it. To learn about the specifics, you need to look at the help file specific for the type of fit object that is being used. The predict is a actually a special type of function in R (called a generic function) that calls other functions depending on what kind of object it receives. So if predict receives an object coming out of the lm function, it will call predict.lm. If it receives an object coming out of glm, it calls predict.glm. These two functions are similar but different. You can learn more about the differences by reading the help files: ?predict.lm  and ?predict.glm


67.2 Regression for a categorical outcome
The regression approach can also be applied to categorical data. To illustrate this, we will apply it to our previous predicting sex example:
If we define the outcome  Y as 1 for females and 0 for males, and X as the height, in this case we are interested in the conditional probability:

As an example, let’s provide a prediction for a student that is 66 inches tall. What is the conditional probability of being female if you are 66 inches tall? In our dataset we can estimate this by rounding to the nearest inch and computing:

In [ ]:
train_set %>% 
  filter(round(height)==66) %>%
  summarize(mean(sex=="Female"))

We will define  Y=1 for females and  Y=0 for males. To construct a prediction algorithm, we want to estimate the proportion of the population that is female for any given height  
X = x, which we write as the conditional probability described above:  Pr(Y=1|X=x). Let’s see what this looks like for several values of  x (we will remove values of  x with few data points):

In [ ]:
heights %>% 
  mutate(x = round(height)) %>%
  group_by(x) %>%
  filter(n() >= 10) %>%
  summarize(prop = mean(sex == "Female")) %>%
  ggplot(aes(x, prop)) +
  geom_point()

Since the results from the plot above look close to linear, and it is the only approach we currently know, we will try regression. We assume that: p(x)=Pr(Y=1|X=x)=β0+β1x
Note: because  p0(x)=1−p1(x), we will only estimate  p1(x) and drop the index. If we convert the factors to 0s and 1s, we can we can estimate  β0 and  β1 with least squares.

In [ ]:
lm_fit <- mutate(train_set, y = as.numeric(sex == "Female")) %>%
                lm(y ~ height, data = .)

Once we have estimates β0 and β1, we can obtain an actual prediction. Our estimate of the conditional probability p(x) is: p(x)=β0+β1xp(x)=β0+β1x
To form a prediction we define a decision rule: predict female if p(x)>0.5p(x)>0.5. We can compare our predictions to the outcomes using:

In [ ]:
p_hat <- predict(lm_fit, test_set)
y_hat <- ifelse(p_hat > 0.5, "Female", "Male") %>% factor()
confusionMatrix(y_hat, test_set$sex)

We see this method does substantially better than guessing.

**Chapter 69 Logistic regression**
The regression approach can be extended to categorical data. In this chapter we first illustrate how, for binary data, one can simply assign numeric values of 0 and 1 to the outcomes  y, and apply regression as if the data were continuous. We will then point out a limitation with this approach and introduce logistic regression as a solution. Logistic regression is specific case of a set of generalized linear models.

**69.1 Linear regression for a binary outcome**
To illustrate this, we will apply it to our previous predicting sex example:

If we define the outcome  Y as 1 for females and 0 for males, and  X as the height, in this case we are interested in the conditional probability:

Pr(Y=1∣X=x)
 
As an example, let’s provide a prediction for a student that is 66 inches tall. What is the conditional probability of being female if you are 66 inches tall? In our dataset, we can estimate this by rounding to the nearest inch and computing:



In [ ]:
library(dplyr)
train_set %>% 
  filter(round(height)==66) %>%
  summarize(mean(sex=="Female"))

In [ ]:
library(dslabs)
data("olive")
table(olive$region)

In [ ]:
#We remove the area column because we won’t use it as a predictor.
olive <- select(olive, -area)

In [ ]:
library(caret)
fit <- train(region ~ .,  method = "knn", tuneGrid = data.frame(k = seq(1, 15, 2)), data = olive)
ggplot(fit)

In [ ]:
library(tidyr)
olive %>% gather(fatty_acid, percentage, -region) %>%
  ggplot(aes(region, percentage, fill = region)) +
  geom_boxplot() +
  facet_wrap(~fatty_acid, scales = "free")

In [ ]:
p <- olive %>% 
  ggplot(aes(eicosenoic, linoleic, color = region)) + 
  geom_point()


p + geom_vline(xintercept = 0.065, lty = 2) + 
  geom_segment(x = -0.2, y = 10.535, xend = 0.065, yend = 10.535, color = "black", lty = 2)

***Chapter 70 Case study: is it a 2 or a 7?***

In the two simple examples above, we only had one predictor. We actually do not consider these machine learning challenges, which are characterized by cases with many predictors. Let’s go back to the digits example in which we had 784 predictors. For illustrative purposes, we will start by simplifying this problem to one with two predictors and two classes. Specifically, we define the challenge as building an algorithm that can determine if a digit is a 2 or 7 from the predictors. We are not quite ready to build algorithms with 784 predictors so we will extract two simple predictors from the 784: the proportion of dark pixels that are in the upper left quadrant (X1) and the lower right quadrant ( X2).

We then select a random sample of 1,000 digits, 500 in the training set and 500 in the test set and provide them here:

In [ ]:
library(dslabs)
library(dplyr)
library(ggplot2)
data("mnist_27")

We can explore this data by plotting the two predictors and by using colors to denote the labels:

In [ ]:
mnist_27$train %>% ggplot(aes(x_1, x_2, color = y)) +
  geom_point()

We can immediately see some patterns. For example, if  
X1 (the upper left panel) is very large, then the digit is probably a 7. Also, for smaller values of  
X1, the 2s appear to be in the mid range values of  X2.

These are the images of the digits with the largest and smallest values for  X1:

In [ ]:
fit <- glm(y ~ x_1 + x_2, data=mnist_27$train, family="binomial")


We can now build a decision rule based on the estimate of  ^p(x1,x2):

In [ ]:
p_hat <- predict(fit, newdata = mnist_27$test)
y_hat <- factor(ifelse(p_hat > 0.5, 7, 2))
library(caret)
confusionMatrix(data = y_hat, reference = mnist_27$test$y)

We get an accuracy of 0.79! Not bad for our first try. But can we do better?

Because we constructed the mnist_27 example and we had at our disposal 60,000 digits in just the MNIST dataset, we used this to build the true conditional distribution  
p(x1,x2). Keep in mind that this is something we don’t have access to in practice, but we include it in this example because it lets us compare  
^p(x1,x2) to the true  p(x1,x2), which teaches us the limitations of different algorithms. Let’s do that here. We can access and plot  p(x1,x2) like this:

In [ ]:
mnist_27$true_p %>% ggplot(aes(x_1, x_2, fill=p)) +
  geom_raster() 

We will choose better colors and draw a curve that separates pairs  (x1,x2) for which  p(x1,x2)>0.5 and cases for which  p(x1,x2)<0.5:

In [ ]:
mnist_27$true_p %>% ggplot(aes(x_1, x_2, z=p, fill=p)) +
  geom_raster() +
  scale_fill_gradientn(colors=c("#F8766D","white","#00BFC4")) +
  stat_contour(breaks=c(0.5),color="black")

So above you see a plot of the true  p(x,y) . To start understanding the limitations of logistic regression here, first, note that with logistic regression  ^p(x,y) has to be a plane and, as a result, the boundary defined by the decision rule is given by:  ^p(x,y) = 0.5 which implies the boundary can’t be anything other than a straight line:

This implies that our logistic regression approach has no chance of capturing the non-linear nature

In [ ]:
p_hat <- predict(fit, newdata = mnist_27$true_p)
mnist_27$true_p %>% mutate(p_hat = p_hat) %>%
  ggplot(aes(x_1, x_2,  z=p_hat, fill=p_hat)) +
  geom_raster() +
  scale_fill_gradientn(colors=c("#F8766D","white","#00BFC4")) +
  stat_contour(breaks=c(0.5),color="black") 

We can see where the mistakes were made mainly come from low values  x1 that have either high or low value of  x2. Logistic regression can’t catch this.

In [ ]:
p_hat <- predict(fit, newdata = mnist_27$true_p)
mnist_27$true_p %>% mutate(p_hat = p_hat) %>%
  ggplot() +
  stat_contour(aes(x_1, x_2, z=p_hat), breaks=c(0.5), color="black") + 
  geom_point(mapping = aes(x_1, x_2, color=y), data = mnist_27$test) 

We need something more flexible. A method that permits estimates with shapes other than a plane.

We are going to learn a few new algorithms based on different ideas and concepts. But what they all have in common is that they permit more flexible approaches. We will start by describing nearest neighbor and kernel approaches. To introduce the concepts behinds these approaches, we will again start with a simple one dimensional example and describe the concept of smoothing.

**80.3 Regression tree**
When the outcome is continuous, we call this method regression trees. We will use a continuous case, the 2008 poll data introduced earlier, to describe the basic idea of how we build these algorithms. We will try to estimate the conditional expectation  
f(x)=E(Y|X=x) with  Y the poll margin and  x the day.

In [ ]:
data("polls_2008")
qplot(day, margin, data = polls_2008)

In [ ]:
library(rpart)
fit <- rpart(margin ~ ., data = polls_2008)
fit

In [ ]:
fit

In [ ]:
plot(fit, margin = 0.2)
text(fit, cex = 0.7)

In [ ]:
polls_2008 %>% 
  mutate(y_hat = predict(fit)) %>% 
  ggplot() +
  geom_point(aes(day, margin)) +
  geom_step(aes(day, y_hat), col="red")

**Playing around**

In [ ]:
head(polls_2008)

In [ ]:
data("polls_2008")
qplot(day, margin, data = polls_2008)

In [ ]:
mnist <- read_mnist()

In [ ]:
a <- lm(margin~day, data = polls_2008)
a$residual

In [ ]:
library(dslabs)
library(dplyr)
library(lubridate)

data("reported_heights")

dat <- mutate(reported_heights, date_time = ymd_hms(time_stamp)) %>%
  filter(date_time >= make_date(2016, 01, 25) & date_time < make_date(2016, 02, 1)) %>%
  mutate(type = ifelse(day(date_time) == 25 & hour(date_time) == 8 & between(minute(date_time), 15, 30), "inclass","online")) %>%
  select(sex, type)

y <- factor(dat$sex, c("Female", "Male"))
x <- dat$type

In [ ]:
dat %>% 
  group_by(type) %>%
  summarize(prop = mean(sex == "Female")) 

In [ ]:
library(caret)
data(iris)
iris <- iris[-which(iris$Species=='setosa'),]
y <- iris$Species

In [ ]:
set.seed(2)
# line of code
test_index <- createDataPartition(y,times=1,p=0.5,list=FALSE) 
test <- iris[test_index,]
train <- iris[-test_index,]

Finding which feature provide the best outcome
How to interprete the foo function  below 

1. normal function with input of x
2. work out the range and put it to rangedvalues with increment of 0.1
3. now, apply the values from range to another function i using sapply
4. 

In [ ]:
head(train,10)

In [ ]:
rangedValues <- seq(range(train$Petal.Width)[1],range(train$Petal.Width)[2],by=0.1)
rangedValues
summary(train$Petal.Width)

In [ ]:
length(train$Petal.Width)
rangedValues

In [ ]:
head(test)

from results below you can see if picks up all rows in a column and run against the ranged, in this case the lowest is 4.9 and run against every single row. 

In [ ]:
apply(test[,c(1,2)],2,foo)

#test[,c(1,2)]

In [ ]:
predictions


In [ ]:
seq(range(train$Sepal.Length)[1],range(train$Sepal.Length)[2],by=0.1)
min(train$Sepal.Length)
max(train$Sepal.Length)

In [ ]:
head(train,10)

In [ ]:
foo <- function(x){
    rangedValues <- seq(range(x)[1],range(x)[2],by=0.1)
    sapply(rangedValues,function(i){
    y_hat <- ifelse(x>i,'virginica','versicolor')
    mean(y_hat==train$Species)})
}
predictions <- apply(test[,-5],2,foo)
sapply(predictions,max)



In [ ]:
b <- train[,-5]


myfn <- function(y){
    range <- seq(range(y)[1],range(y)[2],by=0.1)
    sapply(range,function(x){   
    y_hat <- ifelse(x>y,'versicolor','virginica')
    mean(y_hat==train$Species)
  }) 
}

forecast <- apply(b,2,myfn)
sapply(forecast,max)
sapply(forecast,which.max)




In [ ]:
a <- seq(range(train$Petal.Length)[1],range(train$Petal.Length)[2],by=0.1)
b <- seq(range(train$Petal.Width)[1],range(train$Petal.Width)[2],by=0.1)
a[19]
b[7]

In [ ]:
library(caret)
data(iris)
iris <- iris[-which(iris$Species=='setosa'),]
y <- iris$Species

plot(iris,pch=21,bg=iris$Species)

set.seed(2)
test_index <- createDataPartition(y,times=1,p=0.5,list=FALSE)
test <- iris[test_index,]
train <- iris[-test_index,]

petalLengthRange <- seq(range(train[,3])[1],range(train[,3])[2],by=0.1)
petalWidthRange <- seq(range(train[,4])[1],range(train[,4])[2],by=0.1)
cutoffs <- expand.grid(petalLengthRange,petalWidthRange)

id <- sapply(seq(nrow(cutoffs)),function(i){
y_hat <- ifelse(train[,3]>cutoffs[i,1] | train[,4]>cutoffs[i,2],'virginica','versicolor')
mean(y_hat==train$Species)
}) %>% which.max

optimalCutoff <- cutoffs[id,] %>% as.numeric
y_hat <- ifelse(test[,3]>optimalCutoff[1] & test[,4]>optimalCutoff[2],'virginica','versicolor')
mean(y_hat==test$Species)

In [ ]:

 y_hat <- ifelse(train$Petal.Length < 4.8 | train$Petal.Width < 1.6, "versicolor", "virginica") %>% factor(levels = levels(test$Species))
confusionMatrix(y_hat,test$Species)

In [ ]:
forecast

In [ ]:
foo <- function(x){
    rangedValues <- seq(range(x)[1],range(x)[2],by=0.1)
    sapply(rangedValues,function(i){
    y_hat <- ifelse(x>i,'virginica','versicolor')
    mean(y_hat==train$Species)})
}
predictions <- apply(test[,-5],2,foo)
sapply(predictions,max)


**Cut off formula (Original)**

In [ ]:
cutoff <- seq(61, 70)
accuracy <- sapply(cutoff, function(x){
  y_hat <- ifelse(train_set$height > x, "Male", "Female") %>% factor(levels = levels(test_set$sex))
  mean(y_hat == train_set$sex)
})


max(accuracy)

best_cutoff <- cutoff[which.max(accuracy)]
best_cutoff


y_hat <- ifelse(test_set$height > best_cutoff, "Male", "Female") %>% factor(levels = levels(test_set$sex))
y_hat <- factor(y_hat)
mean(y_hat == test_set$sex)

In [ ]:
summary(train)

In [ ]:
cutoff <- seq(1, 2.5)
accuracy <- sapply(cutoff, function(x){
  y_hat <- ifelse(train$Petal.Width > x, "versicolor", "virginica") %>% factor(levels = levels(test$Species))
  mean(y_hat == train$Species)
})

max(accuracy)

best_cutoff <- cutoff[which.max(accuracy)]
best_cutoff

accuracy

In [ ]:
cutoff <- seq(3, 6.9)
accuracy <- sapply(cutoff, function(x){
  y_hat <- ifelse(train$Petal.Length < x, "versicolor", "virginica") %>% factor(levels = levels(test$Species))
  mean(y_hat == train$Species)
})

max(accuracy)

best_cutoff <- cutoff[which.max(accuracy)]
best_cutoff


In [ ]:
 y_hat <- ifelse(test$Petal.Length < 5, "versicolor", "virginica") %>% factor(levels = levels(test$Species))
confusionMatrix(y_hat,test$Species)

In [ ]:
# combined petal length and width
train$x <- train$Petal.Length + train$Petal.Width
head(train)

In [ ]:
summary(train)

In [ ]:
accuracy
best_cutoff

In [ ]:
library(dplyr)
library(purrr)
cutoff <- seq(3, 6.9, by=0.1)
accuracy <- map_dbl(cutoff, function(x){
  y_hat <- ifelse(train$Petal.Length < x, "versicolor", "virginica") %>% factor(levels = levels(test$Species))
  mean(y_hat == train$Species)
})

max(accuracy)

best_cutoff <- cutoff[which.max(accuracy)]
best_cutoff

plot(cutoff,accuracy,type='o')

 y_hat <- ifelse(train$Petal.Length < best_cutoff, "versicolor", "virginica") %>% factor(levels = levels(test$Species))
confusionMatrix(y_hat,test$Species)

In [ ]:
library(dplyr)
library(purrr)
cutoff <- seq(1, 6.9)
accuracy <- map_dbl(cutoff, function(x){
  y_hat <- ifelse(train$Petal.Length > x | train$Petal.Width > x, "versicolor", "virginica") %>% factor(levels = levels(test$Species))
  mean(y_hat == train$Species)
})

max(accuracy)

best_cutoff <- cutoff[which.max(accuracy)]
best_cutoff

 y_hat <- ifelse(train$Petal.Length < best_cutoff | train$Petal.Width < best_cutoff, "versicolor", "virginica") %>% factor(levels = levels(test$Species))
confusionMatrix(y_hat,test$Species)

**CONDITIONAL PROBABILITY**

In a previous module, we covered Bayes' theorem and the Bayesian paradigm. Conditional probabilities are a fundamental part of this previous covered rule.

We first review a simple example to go over conditional probabilities.

Assume a patient comes into the doctor’s office to test whether they have a particular disease.

* The disease is prevalent in about 2% of the community: p(disease)
* p(healthy) : 100% - 2% = 98%
* If you split disease down, then prob is 85% positive and 15% negative --> p( positive|disease) = 85%   &  p( negative|disease) = 15% 
* If you split healthy down, then prob is 10% positive and 90% negative --> p( positive|healthy) = 10%    &  p( negative|healthy) = 90% 

You can then broken down these to numbers to make sense how the probability is calculated. 

E.g total is 1,000,000 population

* p(healthy) = 98% * 1,000,000 = 980,000
* p(disease) = 2% * 1,000,000 =      20,000

* p(positive|healthy) = 98% * 1,000,000 * 10%   =     98,000  -->** but when they say p(positive|healthy) they don't take account the 98% just 10%. same with below ****
* p(negative|healthy) = 98% * 1,000,000 * 90% =   882,000

* p(positive|disease) =  2% * 1,000,000 * 85%    =     17,000
* p(negative|disease) = 2% * 1,000,000 * 15%    =       3,000 

so the following calculation will make sense based on the above
* p(positive)  = (98% * 10%) + (2%*85%)
* p(negative) = (98%*90%) + (98%*15%)
* p(disease|positive) is these values  17,000/(17,000+98,000) = 0.147826086956522

if you use textbook it is the same as p(disease)/p(positive) * p(positive|disease)

Below will explain the logic on how it gets to that 

17,000/(17,000+98,000) = p(positive|disease) * p(disease) /** ( p(positive|disease) * p(disease) ) + ( p(positive|healthy) * p(healthy))**

And we know the one below in bold = p(positive)

So the final equation :
p(disease|positive) = p(positive|disease) * p(disease)/p(positive)

The following 4 questions (Q2-Q5) all relate to implementing this calculation using R.

We have a hypothetical population of 1 million individuals with the following conditional probabilities as described below:

The test is positive 85% of the time when tested on a patient with the disease (high sensitivity): 
The test is negative 90% of the time when tested on a healthy patient (high specificity): 
The disease is prevalent in about 2% of the community: 
Here is some sample code to get you started:

set.seed(1)
disease <- sample(c(0,1), size=1e6, replace=TRUE, prob=c(0.98,0.02))
test <- rep(NA, 1e6)
test[disease==0] <- sample(c(0,1), size=sum(disease==0), replace=TRUE, prob=c(0.90,0.10))
test[disease==1] <- sample(c(0,1), size=sum(disease==1), replace=TRUE, prob=c(0.15, 0.85))

In [ ]:
P(disease|test-) = P(Disease)/P(test-) * P(test-|disease)

     = 0.02*0.15/0.885

In [ ]:
p(disease)

In [ ]:
set.seed(1)
disease <- sample(c(0,1), size=1e6, replace=TRUE, prob=c(0.98,0.02))
test <- rep(NA, 1e6)
test[disease==0] <- sample(c(0,1), size=sum(disease==0), replace=TRUE, prob=c(0.90,0.10))
test[disease==1] <- sample(c(0,1), size=sum(disease==1), replace=TRUE, prob=c(0.15, 0.85))

In [ ]:
sum(test==0)
sum(disease==0)

In [ ]:

mean(disease[test==0])

In [ ]:
mean(disease[test==1]==1)

In [ ]:
(sum(a==0)-sum(b==0))/sum(b==0)


In [ ]:
a <- c(1,1,0,0,1,0,0,0)
b <- c(1,0,1,0,0,1,0,1)
mean(b[a==0])
mean(disease==1)

In [ ]:
set.seed(1)
disease <- sample(c(0,1), size=20, replace=TRUE, prob=c(0.98,0.02))
test <- rep(NA, 20)
test[disease==0] <- sample(c(0,1), size=sum(disease==0), replace=TRUE, prob=c(0.90,0.10))
test[disease==1] <- sample(c(0,1), size=sum(disease==1), replace=TRUE, prob=c(0.15, 0.85))


In [ ]:
#RR = P(disease | test+) / P(disease)

0.1478261/0.019918

In [ ]:
sum(disease)/length(disease)

In [ ]:
library(dplyr)
test %>% summarize(mean(disease))

In [ ]:
heights$true

In [ ]:
library(dplyr)
library(dslabs)
data("heights")
library("ggplot2")


heights %>% 
mutate(height = round(height)) %>%
group_by(height) %>%
summarize(p = mean(sex == "Male")) %>%


qplot(height, p, data =.)



In [ ]:
library(dplyr)
library(dslabs)
data("heights")
library("ggplot2")


heights %>% 
mutate(height = round(height)) %>%
group_by(height) %>%
summarize(p = mean(sex == "Male")) %>%


qplot(height, p, data =.)


cut(x, quantile(x, seq(0, 1, 0.1)), include.lowest = TRUE)

In [ ]:

#q1 
library(dplyr)
library(caret)
set.seed(1)
n <- 100
Sigma <- 9*matrix(c(1.0, 0.5, 0.5, 1.0), 2, 2)
dat <- MASS::mvrnorm(n = 100, c(69, 69), Sigma) %>%
data.frame() %>% setNames(c("x", "y"))

In [ ]:
set.seed(1)
index <- createDataPartition(dat$y, times = 1, p = 0.5, list = FALSE)

In [ ]:
train <- dat %>% slice(-index)
test <- dat %>% slice(index)

In [ ]:
fit <- lm(y ~ x, data = train)
fit$coef

In [ ]:
y_hat <- predict(fit, test)
RMSE <- sqrt(mean((y_hat - test$y)^2))
RMSE

In [ ]:
library(caret)
set.seed(1)
b <- replicate(100,{
    index <- createDataPartition(dat$y, times = 1, p = 0.5, list = FALSE)
    train <- dat %>% slice(-index)
    test <- dat %>% slice(index)
    fit <- lm(y ~ x, data = train)
    y_hat <- predict(fit, test)
    RMSE <- sqrt(mean((y_hat - test$y)^2))
    RMSE})


In [ ]:
mean(b)
sd(b)

In [ ]:
packageVersion("dslabs")

In [ ]:
update.packages("dslabs")

In [ ]:
c <- function(n){replicate(n,{
    index <- createDataPartition(dat$y, times = 1, p = 0.5, list = FALSE)
    train <- dat %>% slice(-index)
    test <- dat %>% slice(index)
    fit <- lm(y ~ x, data = train)
    y_hat <- predict(fit, test)
    RMSE <- sqrt(mean((y_hat - test$y)^2))
    RMSE})
h <- c(mean(b),sd(b))
h}

In [ ]:
N <- c(100,200,300)
sapply(N,c)

In [ ]:
x <- c(1, 5, 4, 9, 0)

In [ ]:
x <- c(1,2,3,4)
sapply(x,function(x) x*2)


In [ ]:
B <- 10000
tallest <- replicate(B, {
  simulated_data <- rnorm(800, avg, s)
  max(simulated_data)
})

In [ ]:
gg <- c(1,2,3)

In [ ]:
sapply(1:3, function(x) x^2)

In [ ]:
myvector <- 1:15
a <- matrix(myvector,5,3)
b <- matrix(myvector,5,3,byrow=TRUE)
rowSums(a)
rowMeans(a)
colSums(a)
colMeans(a)
a

In [ ]:
x <- matrix(rnorm(100*10), 5, 2)
x

In [ ]:
seq(nrow(x)) 
1:nrow(x)
sweep(x, 2, 1:nrow(x),"+")

In [ ]:
install.packages("pdftools")

In [ ]:
library(tidyverse)
library(purrr)
library(pdftools)
    
fn <- system.file("extdata", "RD-Mortality-Report_2015-18-180531.pdf", package="dslabs")
dat <- map_df(str_split(pdf_text(fn), "\n"), function(s){
s <- str_trim(s)
header_index <- str_which(s, "2015")[1]
tmp <- str_split(s[header_index], "\\s+", simplify = TRUE)
month <- tmp[1]
header <- tmp[-1]
tail_index  <- str_which(s, "Total")
n <- str_count(s, "\\d+")
out <- c(1:header_index, which(n==1), which(n>=28), tail_index:length(s))
s[-out] %>%
str_remove_all("[^\\d\\s]") %>%
str_trim() %>%
str_split_fixed("\\s+", n = 6) %>%
.[,1:5] %>%
as_data_frame() %>% 
setNames(c("day", header)) %>%
mutate(month = month,
day = as.numeric(day)) %>%
gather(year, deaths, -c(day, month)) %>%
mutate(deaths = as.numeric(deaths))
}) %>%
mutate(month = recode(month, "JAN" = 1, "FEB" = 2, "MAR" = 3, "APR" = 4, "MAY" = 5, "JUN" = 6, 
                          "JUL" = 7, "AGO" = 8, "SEP" = 9, "OCT" = 10, "NOV" = 11, "DEC" = 12)) %>%
mutate(date = make_date(year, month, day)) %>%
filter(date <= "2018-05-01")

In [ ]:
library(dslabs)
if(!exists("mnist")) mnist <- read_mnist()


In [ ]:
class(mnist$train$images)

In [ ]:
install.packages("dslabs")
system.file("extdata", package="dslabs")
path <- system.file("extdata", package="dslabs")
list.files(path)

In [ ]:
library(caret)
library(dslabs)
fit_glm <- glm(y ~ x_1 + x_2, data=mnist_27$train, family="binomial")
p_hat_logistic <- predict(fit_glm,mnist_27$test)
y_hat_logistic <- factor(ifelse(p_hat_logistic>0.5,7,2))
confusionMatrix(data = y_hat_logistic, reference=mnist_27$test$y)